In [2]:
import pandas as pd

In [4]:
df = pd.read_csv('biv_clustered_knn.csv', encoding='cp1251', sep=',', low_memory=False)
pd.set_option('display.max_columns', None)
df.drop('Unnamed: 0', axis= 1 , inplace= True )
df.head(5)

,uid,name,category1_name,category2_name,category3_name,category4_name,clusters
0,00e7d508-7577-11e9-80ca-901b0e95a2a8,Тент ENO ProFly XL Sil Lichen,Палатки,Тенты и противомоскитные пологи,NaN,NaN,95
1,00e7d508-7577-11e9-80ca-901b0e95a2a8,Тент ENO ProFly XL Sil Lichen,Палатки,Тенты и противомоскитные пологи,NaN,NaN,95
2,00e7d508-7577-11e9-80ca-901b0e95a2a8,Тент ENO ProFly XL Sil Lichen,Палатки,Тенты и противомоскитные пологи,NaN,NaN,95
3,00e7d508-7577-11e9-80ca-901b0e95a2a8,Тент ENO ProFly XL Sil Lichen,Палатки,Тенты и противомоскитные пологи,NaN,NaN,95
4,00e7d508-7577-11e9-80ca-901b0e95a2a8,Тент ENO ProFly XL Sil Lichen,Палатки,Тенты и противомоскитные пологи,NaN,NaN,71


In [6]:
# Группировка данных по кластерам
cluster_groups = df.groupby('clusters')['uid'].apply(list).reset_index()

In [8]:
cluster_groups

,clusters,uid
0,0,"[0db365fa-9801-11ed-85da-b42e99c9b482, 05e8542..."
1,1,"[0a8300e5-850c-11ed-85da-b42e99c9b482, 04a81cd..."
2,2,"[294d2d79-478c-11e7-80c8-901b0e95a2a8, 9809c8f..."
3,3,"[0289558e-df98-11e5-80ca-002590d0f723, 0501704..."
4,4,"[23e7c1f1-3c3c-11e9-80ca-901b0e95a2a8, 66fb464..."
...,...,...
351,351,"[09afed68-243b-11e7-80c7-901b0e95a2a8, 2cb3e14..."
352,352,"[03b14df9-be62-11ed-85da-b42e99c9b482, 042d3b9..."
353,353,[2c277585-18b0-11ed-85da-b42e99c9b482]
354,354,"[04a81cec-18b0-11ed-85da-b42e99c9b482, 04a81ce..."


In [10]:
# Создание словаря с соответствием ID товара и списком товаров в одном кластере
product_clusters = {}
for _, row in cluster_groups.iterrows():
    cluster_id = row['clusters']
    product_ids = row['uid']
    for product_id in product_ids:
        if product_id not in product_clusters:
            product_clusters[product_id] = []
        for pid in product_ids:
            if pid != product_id and pid not in product_clusters[product_id]:
                product_clusters[product_id].append(pid)


In [14]:
product_clusters.update({k: set(v) for k, v in product_clusters.items()})

In [21]:
#product_clusters

In [16]:
import chardet
file = 'разметка без стат.csv'
with open(file, 'rb') as rawdata:
    result = chardet.detect(rawdata.read(100000))
result

{'encoding': 'windows-1251',
 'confidence': 0.9241820537703935,
 'language': 'Russian'}

In [18]:
df_markup = pd.read_csv('разметка без стат.csv', encoding='cp1251', sep=',', low_memory=False)
pd.set_option('display.max_columns', None)
df_markup.drop('Unnamed: 0', axis= 1 , inplace= True )
df_markup.head(5)

,target_cat2_name,add1,add2,add3,add4,add5,add6,add7,add8
0,Туристические газовые горелки,Газовые баллоны,Чайники и кофеварки,Наборы туристической посуды,Сковороды,Аксессуары к горелкам и лампам,Туристические палатки,Туристические рюкзаки,Туристические спальные мешки
1,Газовые баллоны,Туристические газовые горелки,Наборы туристической посуды,Туристические палатки,Туристические рюкзаки,Туристические спальные мешки,Миски,Чашки,Вилки и ложки
2,Аксессуары к горелкам и лампам,Туристические газовые горелки,Наборы туристической посуды,Туристические фонари,Туристические рюкзаки,Кухонные принадлежности,Миски,Чашки,Вилки и ложки
3,Жидкотопливные и мультитопливные горелки,Наборы туристической посуды,Чайники и кофеварки,Сковороды,Кухонные принадлежности,Аксессуары к горелкам и лампам,Экстремальные спальные мешки,Экстремальные палатки,Утепленная одежда
4,Газовые плиты,Газовые баллоны,Наборы туристической посуды,Сковороды,Кухонные принадлежности,Кемпинговые палатки,Столы для кемпинга,Стулья и кресла для кемпинга,Туристические фонари


In [20]:
# Создаем словарь по разметке
murkup_dict = {}
for index, row in df_markup.iterrows():
    key = row['target_cat2_name']
    values = row.drop('target_cat2_name').tolist()
    murkup_dict[key] = values

In [9]:
# murkup_dict

In [22]:
# Создаем список целевых категорий, с вхождением в которые потом будем сравнивать ключи нашего словаря кластеров
target_cat = df_markup['target_cat2_name'].tolist()
#target_cat

In [12]:
#target_cat список целевых категорий
#murkup_dict словарь разметки
#product_clusters словарь uid : список сопутствующих uid
#df 


In [ ]:
#Может быть тут перед фильтрацией сделать лишнее преобразование list(set(list)) для каждого списка значений??? 

In [24]:
# Фильтрация товаров по категориям
#result = {}
filtered_product_clusters = {}
for cluster, products in product_clusters.items():
    cluster_cat2 = df.loc[df['uid'] == cluster, 'category2_name'].iloc[0]
    # Проверяем, что cluster_cat2 не равно None
    if cluster_cat2 is not None:
        # Проверяем, что ключ словаря - товар из целевой категории
        if cluster_cat2 in target_cat:
            filtered_products = []
            for product in products:
                # Проверяем, что значения для ключа - товары из размеченных категорий 
                # Получаем список названий категорий элемента из значений
                prod_categories = df.loc[df['uid'] == product, 'category1_name':'category4_name'].values.flatten().tolist()
                # Убиваем все нулевые уровни
                prod_categories = list(filter(None, prod_categories))
                # Сравниваем их с размеченными 
                for category in prod_categories:
                    # Перебираем все оставшиеся ненулевые уровни иерархии 
                    if category in murkup_dict.get(cluster_cat2):        
                        filtered_products.append(product)
            filtered_product_clusters[cluster] = filtered_products
    else:
        print(f"Для кластера {cluster} не найдена категория")


In [27]:
# Преобразование в JSON
import json
json_data = json.dumps(filtered_product_clusters, indent=4)

# Запись JSON в файл
with open('product_clusters_knn.json', 'w') as json_file:
    json_file.write(json_data)


считал без 5 минут 2 часа(((((((

проверка на вшивость всего файла. Делаем такой же json, только с названиями товаров и посмотрим внутрь.

target_cat список целевых категорий \n
murkup_dict словарь разметки \n 
product_clusters словарь uid : список сопутствующих uid  \n
df uid-название-иерархия-кластеры \n

In [29]:
# Создаем словарь uid-name
uid_name_dict = {}
for index, row in df.iterrows():
    key = row['uid']
    value = row['name']
    uid_name_dict[key] = value

In [31]:
# Создаем пустой словарь для нового соответствия на основе названий товаров
prod_clusters_names = {}

# Проходим по каждой паре ключ-значение в словаре filtered_product_clusters
for key, values in filtered_product_clusters.items():
    # Находим соответствующее название товара по ключу в словаре uid-name
    product_name = uid_name_dict[key]
    
    # Находим названия товаров для значений
    values_names = [uid_name_dict[value] for value in values]
    
    # Добавляем соответствующую пару ключ-значение в новый словарь prod_clusters_names = {}

    prod_clusters_names[product_name] = values_names


In [35]:
import json
json_data = json.dumps(prod_clusters_names, ensure_ascii = False, indent=4)

# Запись JSON в файл
with open('product_clusters_names.json', 'w') as json_file:
    json_file.write(json_data)


не отрабатывает отбор по отсутствию дубликатов ( последний вложенный цикл в 5 ячейке??? 